In [ ]:
import numpy as np
from skimage import io

from numpy.lib.stride_tricks import sliding_window_view

from matplotlib import pyplot as plt

from matplotlib import cm
import networkx as nx
from tqdm.notebook import tqdm

from bresenham import bresenham
from matplotlib import cm, colors
import matplotlib.patches as mpatches
from collections import Counter
import pandas as pd

from shapely.geometry import Polygon, LineString
import matplotlib.pyplot as plt

import itertools

from mpire import WorkerPool

import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path
import json

from wc_cv.src.utils import grainPreprocess, grainMark, SEMDataset, grainDraw, Crack

from pathos.multiprocessing import ProcessingPool as Pool  # Alternative parallelization

# Shortest paths

In [ ]:
# df['path'] = df['path'].apply(lambda x: np.array(x))
# df.to_csv('shortest_paths_edges.csv')

In [ ]:
# 'path':all_paths,
# 'path_len_edges':all_path_len_edges,
# 'path_len_pixel':all_path_len_pixels,
# 'entry_node':all_entry_nodes,
# 'exit_node':all_exit_nodes

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=1

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    # for i in range(N):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    # for i in range(N):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)

In [ ]:
# df['path'] = df['path'].apply(lambda x: np.array(x))
# df.to_csv('shortest_paths_edges.csv')

df1=pd.read_csv('shortest_paths_edges.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

N=10

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    for i in range(N):
        row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[i]
        shortest_entry_paths.append(row)


df1_shortest_entry = pd.DataFrame(shortest_entry_paths)


In [ ]:
df_shortest_entry

In [ ]:
x=329

df_shortest_entry[df_shortest_entry['path'].apply(lambda arr: x in arr)]

In [ ]:
df=pd.read_csv('paths.csv')

entry_nodes = np.unique(df['entry_node'])
exit_nodes = np.unique(df['exit_node'])

shortest_entry_paths = []
for entry_node in tqdm(entry_nodes):
    row = df[df.entry_node==entry_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_entry_paths.append(row)

shortest_exit_paths = []
for exit_node in tqdm(exit_nodes):
    row = df[df.exit_node==exit_node].sort_values(by='path_len_pixel').iloc[0]
    shortest_exit_paths.append(row)

df_shortest_entry = pd.DataFrame(shortest_entry_paths)
df_shortest_exit = pd.DataFrame(shortest_exit_paths)



fig,axes = plt.subplots(1,1,figsize=(15,15))

pos = nx.get_node_attributes(g, 'pos')

for i in range(len(df_shortest_entry)):
    line = df_shortest_entry.iloc[i]
    path = line['path']
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='blue',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['entry_node']],node_color='blue',node_size = 100)

for i in range(len(df_shortest_exit)):
    line = df_shortest_exit.iloc[i]
    path = list(reversed(line['path']))
    path_edges = list(zip(path,path[1:]))

    nx.draw_networkx_edges(g,pos,edgelist=path_edges,edge_color='r',width=1)
    nx.draw_networkx_nodes(g,pos,nodelist=[line['exit_node']],node_color='r',node_size = 100)

axes.imshow(img_aligned, cmap='gray')

plt.axis("on")
axes.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

axes.arrow(12, border-5, 0, 200, width=0.3, length_includes_head=True, head_width=10, head_length=10,color=(0,0,0))

axes.text(2, img_aligned.shape[0]*2/3,'hit direction',rotation=90, color=(0, 0, 0),fontsize=15)
axes.text(img_aligned.shape[1]/2-25, border/3,'entry nodes', color=(0, 0, 1),fontsize=15)
axes.text(img_aligned.shape[1]/2-25,img_aligned.shape[0] - border/3,'exit nodes', color=(1, 0, 0),fontsize=15)

# plt.savefig('shortests_paths.jpg',bbox_inches='tight')

plt.show()

## Graph properties

In [ ]:
# mean_path_length_edges = df['path_len_edges'].mean()

df=[]
mean_path_length_edges = 1

mean_degree  = np.array(g.degree)[:,1].mean()
entry_nodes_num = len(entry_nodes)
exit_nodes_num = len(exit_nodes)
number_of_nodes = len(g.nodes)

cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

print(f'number of paths {len(df)/1e6:.2}*10^6')
print('-----')
print(f'number of entry nodes n={entry_nodes_num}' )
print(f'number of exit nodes k={exit_nodes_num}')
print(f'mean node degree p={mean_degree:.2f}')
print(f'mean path length edges L={mean_path_length_edges:.2f}')
print(f'number of nodes t={number_of_nodes}')
print(f'number of edges t={len(g.edges)}')
print(f'number  of pairs={len(cart_list)}')

m = (number_of_nodes-entry_nodes_num-exit_nodes_num)/mean_path_length_edges
print(f'm={m:.2f}')

N = entry_nodes_num*exit_nodes_num*(mean_degree/2 * mean_degree/2)**m
print(f'N={N/1e6:.2f}*10^6')

In [ ]:
%%time

mem1 = psutil.virtual_memory().available
def find_paths(entry_node, exit_node):
    all_paths=[]
    all_entry_nodes=[]
    all_exit_nodes=[]
    all_path_len_edges=[]
    all_path_len_pixels=[]
    
    os.system(f'/bin/bash -c "bfs_custom/bfs_paths_dev bfs_custom/200x500.txt {entry_node} {exit_node} ./tmp/" ')
    
    paths = 0
    return 

workers = 200
cart_list=[entry_nodes, exit_nodes]
cart_list=[element for element in itertools.product(*cart_list)]

with WorkerPool(n_jobs=workers) as pool:
    results = pool.map(find_paths, cart_list, progress_bar= True)

# df = pd.concat(results,axis=0)

mem2 = psutil.virtual_memory().available
print('entry_nodes', entry_nodes)
print('exit_nodes', exit_nodes)
print('used memory Gib',(mem1-mem2)/1024**3)

In [ ]:
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:520,:500]
# image = io.imread('WC+8Co_5_fixed_001.jpg')[:300,:800]
image = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-003.jpeg')[:300,:500]

#  некорректно
# image = io.imread('../datasets/original/o_bc_left/Ultra_Co25/Ultra_Co25-001.jpeg')[:200,:200]


entry_nodes, exit_nodes, num_of_nodes, img_aligned, nodes_index2global_nodes_coord, cnts = preprocess_graph_image(image,plot=True)

g = create_crack_graph(img_aligned.shape, num_of_nodes,nodes_index2global_nodes_coord,cnts,  plot=False)

# nx.write_edgelist(g,'350x500.txt',data=False)

# save graph object to file
# pickle.dump(g, open('g.pickle', 'wb'))


#200

# entry_nodes [18, 30, 11, 17, 0, 10]
# exit_nodes [83, 45, 46, 47, 57, 58, 59]

# 250

# entry_nodes [18, 30, 11, 17, 0, 10]
# exit_nodes [81, 82, 83, 78, 79, 74, 75]

# 300

# entry_nodes [17, 28, 11, 16, 0, 10]
# exit_nodes [96, 97, 84, 85, 76, 77]

# 350 

# entry_nodes [17, 28, 11, 16, 0, 10]
# exit_nodes [111, 112, 101, 102, 84, 85]

# plt.imshow(img_aligned)

In [ ]:
!cat /proc/meminfo | grep Mem

In [ ]:
!nvcc -O2 bfs_custom/bfs_paths_dev.cu -o bfs_custom/bfs_paths_dev 
!nvcc -O2 bfs_custom/bfs_paths.cu -o bfs_custom/bfs_paths

In [ ]:
!bfs_custom/bfs_paths bfs_custom/300x500.txt 0 97 ./tmp/

In [ ]:
with open('bfs_paths_15_110_300x500.json') as f:
    data = json.load(f)

In [ ]:
%%time

def find_paths(entry_node, exit_node):
    all_paths=[]
    all_entry_nodes=[]
    all_exit_nodes=[]
    all_path_len_edges=[]
    all_path_len_pixels=[]
    
    paths = list(nx.all_simple_paths(g, source=entry_node, target=exit_node,cutoff=10))

    for path in paths: 
        all_paths.append(path)
        all_entry_nodes.append(int(entry_node))
        all_exit_nodes.append(int(exit_node))
        all_path_len_edges.append(len(path))
        l=nx.path_weight(g,path, 'weight')
        all_path_len_pixels.append(l)

    return pd.DataFrame({'path':all_paths,
                         'path_len_edges':all_path_len_edges,
                         'path_len_pixel':all_path_len_pixels,
                         'entry_node':all_entry_nodes,
                         'exit_node':all_exit_nodes
                        })


paths_num = []
mean_paths_len = []
std_paths_len = [] 

closest_nodes = []

epsilon = 100
workers = 23

pos = nx.get_node_attributes(g, 'pos')

all_paths = []
cart_list = []

for node1,data1 in tqdm(g.nodes(data=True)):
    target_pos = data1['pos']
    closest_nodes.append([])
    for node2, data2 in g.nodes(data=True):
        node_pos = data2['pos']
        distance = math.sqrt((node_pos[0] - target_pos[0])**2 + (node_pos[1] - target_pos[1])**2)
        if distance <= epsilon:
            closest_nodes[node1].append((node2, distance))
            exit_nodes = np.array(closest_nodes[node1])[1:,0]
            tmp_cart_list =[[node1], list(exit_nodes)]
            tmp_cart_list=[element for element in itertools.product(*tmp_cart_list)]
            cart_list.extend(tmp_cart_list)


cart_list_unique = np.unique(cart_list,axis=0)
cart_list_unique =[list(element) for element in cart_list_unique]

with WorkerPool(n_jobs=workers) as pool:
    results = pool.map(find_paths, cart_list_unique, progress_bar = True)

all_paths.append(results)
    


# Shortest paths plot

In [ ]:
df_shortest_entry['path'] = df_shortest_entry['path'].apply(lambda x: np.array(x))
df_shortest_exit['path'] = df_shortest_exit['path'].apply(lambda x: np.array(x))

df_shortest_entry['path_len_pixel'] = df_shortest_entry['path_len_pixel'].apply(lambda x: int(x))
df_shortest_exit['path_len_pixel'] = df_shortest_exit['path_len_pixel'].apply(lambda x: int(x))

df_shortest_entry['entry_node'] = df_shortest_entry['entry_node'].apply(lambda x: int(x))
df_shortest_exit['entry_node'] = df_shortest_exit['entry_node'].apply(lambda x: int(x))

df_shortest_entry['exit_node'] = df_shortest_entry['exit_node'].apply(lambda x: int(x))
df_shortest_exit['exit_node'] = df_shortest_exit['exit_node'].apply(lambda x: int(x))

df_shortest_entry['path_len_edges'] = df_shortest_entry['path_len_edges'].apply(lambda x: int(x))
df_shortest_exit['path_len_edges'] = df_shortest_exit['path_len_edges'].apply(lambda x: int(x))

df_shortest_entry.to_csv('shortest_entry.csv')
df_shortest_exit.to_csv('shortest_exit.csv')

# Plot distances hists

In [ ]:
def plot_paths_dist(data, name, title=False, folder='tmp', save=True, bins = 60, xlim = None):
    std=np.std(data)
    mean=np.mean(data)
    
    plt.figure(figsize=(7,5))
    plt.hist(data, bins=bins)
    
    plt.axvline(mean-2*std,color=(0.2,0.5,0.5))
    plt.axvline(mean-std,color=(1,1,0))
    plt.axvline(mean,color=(1,0,0))
    plt.axvline(mean+std,color=(0,1,1))
    plt.axvline(mean+2*std,color=(0,1,0.5))
    if title:
        plt.title(title)
    plt.legend(['mean-2*std','mean-std','mean','mean+std','mean+2*std',f'paths len\nnumber of paths\n{len(data)/10**6:.3f}*10^6\nmean={mean:.2f}\nstd={std:.2f}'])
    plt.xlabel('length of crack, pixels')
    
    if xlim:
        plt.xlim(xlim[0],xlim[0])
    
    plt.savefig(f'{folder}/{name}', bbox_inches='tight')
    plt.show()


pixel = 50 / 1000

save=False
# save=True

folder = "tmp_plots"
Path(folder).mkdir(parents=True, exist_ok=True)

names = []

# all paths pixels 
data=df['path_len_pixel']
name = f"entry_exit_paths_all_pixels.jpg"
names.append(name)
plot_paths_dist(data, name, folder = folder, save=save)

# exit paths pixels 
for i,exit_node in tqdm(enumerate(exit_nodes)):
    name = f"entry_exit_paths_{i}_pixels.jpg"
    names.append(name)
    data=df[df['exit_node']==exit_nodes[i]]['path_len_pixel']
    plot_paths_dist(data, name,title=f'exit node {exit_node}', folder=folder, save=save)

# all paths edges 
data=df['path_len_edges']
name =  f"entry_exit_paths_all_edges.jpg"
names.append(name)
plot_paths_dist(data, name, folder=folder, save=save, bins=30)

# exit paths edges 
for i,exit_node in tqdm(enumerate(exit_nodes)):
    name = f"entry_exit_paths_{i}_edges.jpg"
    names.append(name)
    data=df[df['exit_node']==exit_nodes[i]]['path_len_edges']
    plot_paths_dist(data, name,title=f'exit node {exit_node}', folder = folder, save=save, bins=30)

shutil.make_archive('plots', 'zip', folder)